In this tutorial, we'll review how to use ``knockpy`` to apply the knockoff framework for selective inference.  

## The basics of knockoffs

In this section, we briefly review the knockoff framework. Users already familiar with knockoffs may want to scroll past this section.

Given a set of $p$ features $X = (X_1, \dots, X_p)$ and an outcome of interest $y$, knockoffs aim to select the small fraction of features on which $y$ actually depends while controlling the false discovery rate. For example, if $y \mid X \sim \mathcal{N}(X \beta, \sigma^2)$ and $\beta$ is sparse, knockoffs aim to identify the set $\{j : \beta_j \ne 0\}$.


Applying the knockoffs framework involves executing three steps.

1. First, we construct synthetic variables $\tilde{X} = (\tilde{X}_1, \dots, \tilde{X}_p)$ called knockoffs. Intuitively, the $j$th knockoff $\tilde{X}_j$ acts as a "negative control" on the $j$th feature $X_j$ during variable selection. In ``knockpy``, knockoffs are denoted as the numpy array ``Xk``. 

2. Second, we use an arbitrary machine learning algorithm -- usually called a *feature statistic* -- to assign variable importances to each of the $p$ features and each of the $p$ knockoffs. For example, we might train a cross-validated Lasso on $[X, \tilde{X}]$ and $y$ and use the lasso coefficient sizes as a measure of variable importance.

3. Intuitively, a non-null feature should be assigned a higher variable importance than its (synthetic) knockoff, whereas knockoffs are constructed such that null features are indistinguishable from their knockoff. The *data-dependent-threshhold* introduced in [Barber and Candes 2015](https://arxiv.org/abs/1404.5609) formalizes this intuition and uses the feature statistics to reject a set of variables such that the expected fraction of false positives is below a prespecified proportion $q$.

There are two main types of knockoffs:

1. **Fixed-X** knockoffs treat the design matrix $X$ as fixed and control the false discovery rate assuming $y \mid X$ follows a homoskedastic gaussian linear response. In this case, it is possible to construct valid knockoffs $\tilde{X}$ with no assumptions on $X$. Note also that when using fixed-X knockoffs, the feature statistic must satisfy a slightly more restrictive *sufficiency* condition (see [Barber and Candes 2015](https://arxiv.org/abs/1404.5609)).

2. **Model-X** knockoffs treat the design matrix $X$ as random. Model-X knockoffs control the false discovery rate for any conditional distribution $y \mid X$, but they crucially assume that the distribution of $X$ is known. Thus, to construct model-X knockoffs, one must know (or at least estimate) the distribution of $X$. 

## The basics of knockpy

The ``knockpy.knockoff_filter.KnockoffFilter`` class in ``knockpy`` generates knockoffs, fits the feature statistics, and applies the data-dependent threshhold all at once. This is demonstrated below.

First we create a synthetic dataset where $X \sim \mathcal{N}(0, \Sigma)$ for some $\Sigma$ and $y \mid X$ Gaussian with homoskedastic errors. The details of this dataset are commented below, but they aren't too important.

In [2]:
# Create a random covariance matrix for X
import numpy as np
import knockpy

np.random.seed(110)
n = 200 # number of data points
p = 500  # number of features
Sigma = knockpy.dgp.AR1(p=p, rho=0.5) # Stationary AR1 process with correlation 0.5

# Sample X
X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma, size=(n,))

# Create random sparse coefficients
beta = knockpy.dgp.create_sparse_coefficients(p=p, sparsity=0.1)
y = np.dot(X, beta) + np.random.randn(n)

Next, we instantiate the ``KnockoffFilter`` class. To do this, we need to specify (i) what type of knockoff sampler we will use and (ii) what feature statistic we are using. Since $X \sim \mathcal{N}(0, \Sigma)$, we will use Gaussian knockoffs, and we'll use the Lasso as our feature statistic, since it's a good all-around choice. We'll explore more options for these arguments later.

In [3]:
from knockpy.knockoff_filter import KnockoffFilter
kfilter = KnockoffFilter(
    ksampler='gaussian',
    fstat='lasso',
)

Next, we run the knockoff filter on our data! Since we are using a model-X approach, we initially pass $\Sigma$ as an input to the knockoff filter.

In [4]:
# Flags of whether each feature was rejected
rejections = kfilter.forward(
    X=X,
    y=y,
    Sigma=Sigma,
    fdr=0.1 # desired level of false discovery rate control
)
# Check the number of discoveries we made
power = np.dot(rejections, beta != 0) / (beta != 0).sum()
fdr = np.dot(rejections, beta == 0) / rejections.sum()
print(f"The knockoff filter has discovered {100*power}% of the non-nulls with an FDR of {100*fdr}%")

The knockoff filter has discovered 68.0% of the non-nulls with an FDR of 19.0476194024086%


Of course, in most real applications, we do not know $\Sigma$. In these cases, the knockoff filter will automatically infer $\Sigma$ using LedoitWolf or GraphicalLasso covariance estimation. Although this invalidates the exact validty of model-X knockoffs, knockoffs have been shown to be fairly robust in this setting.

In [5]:
# Try again with estimated cov matrix
kfilter2 = KnockoffFilter(ksampler='gaussian', fstat='lasso')
rejections = kfilter.forward(X=X, y=y, fdr=0.1)
# Check the number of discoveries we made
power = np.dot(rejections, beta != 0) / (beta != 0).sum()
fdr = np.dot(rejections, beta == 0) / rejections.sum()
print(f"The knockoff filter has discovered {100*power}% of the non-nulls with an FDR of {100*fdr}%")

The knockoff filter has discovered 64.0% of the non-nulls with an FDR of 3.030303120613098%


## Knockoff sampling - Knockoffs galore!

### Metropolized Knockoff Sampling

``knockpy`` implements a fully general covariance-guided Metropolized Knockoff Sampler, which is capable of sampling model-X knockoffs for any $X$ distribution given an (unnormalized) density function of $X$. This metropolized knockoff sampler uses a variety of computational tricks to make it orders of magnitude faster than a naive implementation. 

We review metro in more detail in the advanced usage section. For now, we give an example of the 

In [16]:
import knockpy.metro

# Fake variables for simplicity.
p = 30
X = np.random.randn(n, p)

# An arbitrary log-likelihood function
beta = np.random.randn(p)
def log_likelihood(X):
    X[0:-1]*beta[0:-1]*np.abs(X[1:0])
    
# Undirected graph 
U = np.zeros((p,p))
for xcoord in range(p):
    for offset in [-2, 1, 0, 1, 2]:
        ycoord = min(max(0, xcoord + offset), p-1)
        U[xcoord, ycoord] = 1
        

metrosampler = knockpy.metro.MetropolizedKnockoffSampler(log_likelihood, X=X, undir_graph=U)
Xk = metrosampler.sample_knockoffs()

ValueError: Precision matrix Q is not compatible with undirected graph (nonedge has value 0.2518527691128594)

If you want to build a custom knockoff sampler class, as long as it inherets from the base class ``knockpy.knockoffs.KnockoffSampler``, you can still pass it to the KnockoffFilter constructor to run the knockoff filter. For example, we pass the customized metropolized knockoff sampler to a KnockoffFilter below:

In [15]:
kfilter_metro = KnockoffFilter(ksampler=metrosampler, fstat='ridge')

NameError: name 'metrosampler' is not defined

The metro module can also accept clique potentials from an undirected graphical model in place of the likelihood function to get a $O(p)$ speedup, as discussed in advanced usage.

### Types of knockoffs

There are many ways to generate knockoffs, including "MAC-minimizing" or "SDP" knockoffs, "MVR" knockoffs, "MMI" knockoffs, and "conditional independence" (CI) knockoffs. ``knockpy`` supports all of these knockoff generation methods for "gaussian" and "fixed-X" knockoff types.

In [14]:
# Metropolized sampler for heavy-tailed t markov chain using MVR-guided proposals
kfilter1 = KnockoffFilter(ksampler='artk', knockoff_kwargs={'method':'mvr'})

# This uses gaussian MMI knockoffs
kfilter2 = KnockoffFilter(ksampler='gaussian', knockoff_kwargs={'method':'mmi'})

# This uses fixed-X SDP knockoffs
kfilter3 = KnockoffFilter(ksampler='fx', knockoff_kwargs={'method':'sdp'})

# The 'method' options include: equicorrelated, sdp, mvr, mmi, and ci.

Knockpy provides this functionality by offering very fast solvers for generating the knockoff $S$-matrix, as detailed below:

In [12]:
import time

time0 = time.time()
S_MVR = knockpy.mrc.solve_mvr(Sigma)
time1 = time.time()
mvr_time = np.around(time1 - time0, 1)
S_MMI = knockpy.mrc.solve_mmi(Sigma)
time2 = time.time()
mmi_time = np.around(time2 - time1, 1)
S_SDP = knockpy.mac.solve_SDP(Sigma)
time3 = time.time()
sdp_time = np.around(time3 - time2, 1)
print(f"For p={Sigma.shape[0]}, MVR took {mvr_time} sec, MMI took {mmi_time} sec, SDP took {sdp_time} sec")

For p=500, MVR took 2.6 sec, MMI took 4.6 sec, SDP took 7.4 sec


## Feature Statistics

``knockpy`` offers a whole suite of built-in feature statistics, including cross-validated lasso, ridge, and group-lasso coefficients, lasso-path statistics, the deepPINK statistic [(Lu et. al 2018)](https://arxiv.org/abs/1809.01185), 

### Other Tidbits